In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt
                           aes xsave avx f16c rdrand hypervisor 

**-----------Load The Data------------**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math
import random
import numpy as np
from numpy import ndarray

In [4]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 6 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [5]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/hepmass_ds_mod_orig.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [6]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, MQ)
]

In [12]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


-------------

**--------QuickSelect Algorithm----------------**

In [8]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

---------------

**----------KD Tree---------------------**

In [9]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

**-------KD Tree Node Size Optimization For Rebuild-------**

**--------------Run 1---------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 2_500  # Ensure inclusive range
leaf_size_step = 50

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 50): 100%|██████████| 100/100 [00:00<00:00, 153.97it/s]


-------------------------------------------------------------
Build Time: 20.3970 sec, Search Time: 1.0814 sec
Leaf Size: 50, Total Time: 21.4784 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 100): 100%|██████████| 100/100 [00:00<00:00, 156.28it/s]


-------------------------------------------------------------
Build Time: 19.8174 sec, Search Time: 1.0566 sec
Leaf Size: 100, Total Time: 20.8740 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 15.0635 sec, Search Time: 1.1253 sec
Leaf Size: 2,400, Total Time: 16.1887 sec

49: === Testing with Leaf Size: 2,450, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,450): 100%|██████████| 100/100 [00:00<00:00, 153.29it/s]


-------------------------------------------------------------
Build Time: 15.2848 sec, Search Time: 1.1054 sec
Leaf Size: 2,450, Total Time: 16.3902 sec

50: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 100/100 [00:00<00:00, 152.71it/s]

-------------------------------------------------------------
Build Time: 15.2650 sec, Search Time: 1.1068 sec
Leaf Size: 2,500, Total Time: 16.3718 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 50, Build Time: 20.3970 sec, Search Time: 1.0814 sec, Total Time: 21.4784 sec
Leaf Size: 100, Build Time: 19.8174 sec, Search Time: 1.0566 sec, Total Time: 20.8740 sec
Leaf Size: 150, Build Time: 19.0204 sec, Search Time: 1.0358 sec, Total Time: 20.0562 sec
Leaf Size: 200, Build Time: 18.2996 sec, Search Time: 1.0340 sec, Total Time: 19.3336 sec
Leaf Size: 250, Build Time: 18.0792 sec, Search Time: 1.0275 sec, Total Time: 19.1066 sec
Leaf Size: 300, Build Time: 18.0516 sec, Search Time: 1.0275 sec, Total Time: 19.0792 sec
Leaf Size: 350, Build Time: 17.6962 sec, Search Time: 1.0277 sec, Total Time: 18.7239 sec
Leaf Size: 400, Build Time: 17.0022 sec, Search Time: 1.0087 sec, Total Time: 18.0109 sec
Leaf Size: 450, Build Time: 17.3041 sec, Search Time: 1.0222 sec, Total Time: 18.3263

**-------------Run 2-----------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 2_500  # Ensure inclusive range
leaf_size_step = 50

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 50): 100%|██████████| 100/100 [00:00<00:00, 147.83it/s]


-------------------------------------------------------------
Build Time: 20.9290 sec, Search Time: 1.1143 sec
Leaf Size: 50, Total Time: 22.0432 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 100): 100%|██████████| 100/100 [00:00<00:00, 146.29it/s]


-------------------------------------------------------------
Build Time: 19.8476 sec, Search Time: 1.1121 sec
Leaf Size: 100, Total Time: 20.9597 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 15.2682 sec, Search Time: 1.1150 sec
Leaf Size: 2,400, Total Time: 16.3831 sec

49: === Testing with Leaf Size: 2,450, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,450): 100%|██████████| 100/100 [00:00<00:00, 154.41it/s]


-------------------------------------------------------------
Build Time: 15.2291 sec, Search Time: 1.1001 sec
Leaf Size: 2,450, Total Time: 16.3292 sec

50: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 100/100 [00:00<00:00, 153.90it/s]

-------------------------------------------------------------
Build Time: 14.9763 sec, Search Time: 1.0969 sec
Leaf Size: 2,500, Total Time: 16.0732 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 50, Build Time: 20.9290 sec, Search Time: 1.1143 sec, Total Time: 22.0432 sec
Leaf Size: 100, Build Time: 19.8476 sec, Search Time: 1.1121 sec, Total Time: 20.9597 sec
Leaf Size: 150, Build Time: 19.6692 sec, Search Time: 1.0768 sec, Total Time: 20.7460 sec
Leaf Size: 200, Build Time: 18.9594 sec, Search Time: 1.0701 sec, Total Time: 20.0295 sec
Leaf Size: 250, Build Time: 18.7801 sec, Search Time: 1.0803 sec, Total Time: 19.8603 sec
Leaf Size: 300, Build Time: 18.2004 sec, Search Time: 1.0734 sec, Total Time: 19.2738 sec
Leaf Size: 350, Build Time: 17.8362 sec, Search Time: 1.0541 sec, Total Time: 18.8903 sec
Leaf Size: 400, Build Time: 17.6047 sec, Search Time: 1.0525 sec, Total Time: 18.6572 sec
Leaf Size: 450, Build Time: 17.5948 sec, Search Time: 1.0514 sec, Total Time: 18.6462

**-------------Opt Node Size For Rebuild-------------**

In [ ]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array([  50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700,
  750, 800, 850, 900, 950,1000,1050,1100,1150,1200,1250,1300,1350,1400,
 1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,2050,2100,
 2150,2200,2250,2300,2350,2400,2450,2500])

# Total times from two runs (or versions)
total_times_1 = np.array(
    [21.4784,20.874 ,20.0562,19.3336,19.1066,19.0792,18.7239,18.0109,18.3263,
 18.1995,18.305 ,17.9304,18.3483,17.6225,17.5622,17.6767,17.3597,17.4232,
 17.3814,17.5645,17.3939,17.4662,17.6431,17.0616,17.5543,17.0168,16.8992,
 17.1151,16.569 ,16.6301,16.867 ,16.23  ,16.4297,16.4798,16.2143,16.7079,
 16.3439,16.5093,16.2055,16.3489,16.3908,16.1155,16.554 ,16.3947,16.1224,
 16.2024,16.4158,16.1887,16.3902,16.3718]
    )

total_times_2 = np.array(
   [22.0432,20.9597,20.746 ,20.0295,19.8603,19.2738,18.8903,18.6572,18.6462,
 18.2938,18.5156,18.5501,18.4927,18.0975,17.9326,17.9542,17.6245,17.8088,
 17.9125,17.5882,17.881 ,17.4412,17.4939,17.285 ,17.3889,17.7003,16.8114,
 17.0043,17.3047,16.5841,16.9849,16.8227,16.7681,16.6984,16.7181,16.375 ,
 16.648 ,16.5101,16.8409,16.3929,16.4586,16.4769,16.5888,16.2288,16.2746,
 16.2223,16.2991,16.3831,16.3292,16.0732]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 16.1985
Minimum found at leaf size: 2250


**-----Epsilon Optimization For Rebuild------**

**-----------Run 1-------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size = 2250
# Define epsilon range
epsilon_range_start = 0.05
epsilon_range_end = 0.1  # Ensure inclusive range
epsilon_size_step = 0.001

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.05, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,250): 100%|██████████| 100/100 [00:00<00:00, 111.53it/s]


-------------------------------------------------------------
Build Time: 13.5923 sec, Search Time: 1.3125 sec
Epsilon: 0.05, Total Time: 14.9048 sec

2: === Testing with Epsilon: 0.051, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,250): 100%|██████████| 100/100 [00:00<00:00, 107.90it/s]


-------------------------------------------------------------
Build Time: 13.5257 sec, Search Time: 1.3435 sec
Epsilon: 0.051, Total Time: 14.8692 sec

3: === Testing with Epsilon: 0.052, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 12.5120 sec, Search Time: 1.5084 sec
Epsilon: 0.098, Total Time: 14.0204 sec

50: === Testing with Epsilon: 0.099, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,250): 100%|██████████| 100/100 [00:01<00:00, 87.11it/s]


-------------------------------------------------------------
Build Time: 12.0663 sec, Search Time: 1.5171 sec
Epsilon: 0.099, Total Time: 13.5834 sec

51: === Testing with Epsilon: 0.1, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,250): 100%|██████████| 100/100 [00:01<00:00, 85.27it/s]

-------------------------------------------------------------
Build Time: 12.4612 sec, Search Time: 1.5448 sec
Epsilon: 0.1, Total Time: 14.0060 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.05, Build Time: 13.5923 sec, Search Time: 1.3125 sec, Total Time: 14.9048 sec
Epsilon: 0.051, Build Time: 13.5257 sec, Search Time: 1.3435 sec, Total Time: 14.8692 sec
Epsilon: 0.052, Build Time: 13.3423 sec, Search Time: 1.3031 sec, Total Time: 14.6454 sec
Epsilon: 0.053, Build Time: 13.6716 sec, Search Time: 1.2930 sec, Total Time: 14.9645 sec
Epsilon: 0.054, Build Time: 13.3183 sec, Search Time: 1.3240 sec, Total Time: 14.6423 sec
Epsilon: 0.055, Build Time: 13.1302 sec, Search Time: 1.3307 sec, Total Time: 14.4610 sec
Epsilon: 0.056, Build Time: 13.6066 sec, Search Time: 1.3470 sec, Total Time: 14.9536 sec
Epsilon: 0.057, Build Time: 13.1151 sec, Search Time: 1.3464 sec, Total Time: 14.4615 sec
Epsilon: 0.058, Build Time: 13.3353 sec, Search Time: 1.3458 sec, Total Time: 14.6811 

**----------Run 2--------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size = 2250
# Define epsilon range
epsilon_range_start = 0.05
epsilon_range_end = 0.1  # Ensure inclusive range
epsilon_size_step = 0.001

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.05, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,250): 100%|██████████| 100/100 [00:00<00:00, 113.98it/s]


-------------------------------------------------------------
Build Time: 13.3885 sec, Search Time: 1.2919 sec
Epsilon: 0.05, Total Time: 14.6804 sec

2: === Testing with Epsilon: 0.051, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,250): 100%|██████████| 100/100 [00:00<00:00, 111.03it/s]


-------------------------------------------------------------
Build Time: 13.5706 sec, Search Time: 1.3249 sec
Epsilon: 0.051, Total Time: 14.8955 sec

3: === Testing with Epsilon: 0.052, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 12.4308 sec, Search Time: 1.4915 sec
Epsilon: 0.098, Total Time: 13.9223 sec

50: === Testing with Epsilon: 0.099, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,250): 100%|██████████| 100/100 [00:01<00:00, 86.94it/s]


-------------------------------------------------------------
Build Time: 12.2720 sec, Search Time: 1.5400 sec
Epsilon: 0.099, Total Time: 13.8120 sec

51: === Testing with Epsilon: 0.1, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,250): 100%|██████████| 100/100 [00:01<00:00, 86.77it/s]

-------------------------------------------------------------
Build Time: 12.5124 sec, Search Time: 1.5412 sec
Epsilon: 0.1, Total Time: 14.0537 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.05, Build Time: 13.3885 sec, Search Time: 1.2919 sec, Total Time: 14.6804 sec
Epsilon: 0.051, Build Time: 13.5706 sec, Search Time: 1.3249 sec, Total Time: 14.8955 sec
Epsilon: 0.052, Build Time: 13.8089 sec, Search Time: 1.3401 sec, Total Time: 15.1490 sec
Epsilon: 0.053, Build Time: 13.3714 sec, Search Time: 1.3606 sec, Total Time: 14.7320 sec
Epsilon: 0.054, Build Time: 14.2256 sec, Search Time: 1.3591 sec, Total Time: 15.5846 sec
Epsilon: 0.055, Build Time: 13.5406 sec, Search Time: 1.3696 sec, Total Time: 14.9102 sec
Epsilon: 0.056, Build Time: 13.4853 sec, Search Time: 1.3454 sec, Total Time: 14.8308 sec
Epsilon: 0.057, Build Time: 13.3869 sec, Search Time: 1.4036 sec, Total Time: 14.7905 sec
Epsilon: 0.058, Build Time: 13.6366 sec, Search Time: 1.3460 sec, Total Time: 14.9826 

**--------Epsilon Opt Value For Rebuild------------**

In [ ]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array(
     [0.05 ,0.051,0.052,0.053,0.054,0.055,0.056,0.057,0.058,0.059,0.06 ,0.061,
 0.062,0.063,0.064,0.065,0.066,0.067,0.068,0.069,0.07 ,0.071,0.072,0.073,
 0.074,0.075,0.076,0.077,0.078,0.079,0.08 ,0.081,0.082,0.083,0.084,0.085,
 0.086,0.087,0.088,0.089,0.09 ,0.091,0.092,0.093,0.094,0.095,0.096,0.097,
 0.098,0.099,0.1  ]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
  [14.9048,14.8692,14.6454,14.9645,14.6423,14.461 ,14.9536,14.4615,14.6811,
 14.5677,14.4965,14.3283,14.7837,14.2673,14.4648,14.5066,14.4805,14.5696,
 14.2883,14.5959,14.5748,14.338 ,14.2636,14.2526,14.0965,14.2432,14.2544,
 14.3296,14.1525,14.5019,14.194 ,14.3656,14.0398,14.2281,14.0668,14.0438,
 13.9686,13.9509,13.8663,13.9984,13.8104,14.2145,13.9991,13.8724,13.855 ,
 13.8237,13.7445,13.7924,14.0204,13.5834,14.006 ]
    )

total_times_2 = np.array(
    [14.6804,14.8955,15.149 ,14.732 ,15.5846,14.9102,14.8308,14.7905,14.9826,
 14.4252,14.9347,14.5237,14.9945,14.5202,14.6568,14.6919,14.7635,14.6907,
 14.6065,14.869 ,14.5416,14.6883,14.4298,14.1222,14.4109,14.358 ,14.4864,
 14.4319,14.4585,14.3677,14.552 ,14.1707,14.1524,14.4296,14.1108,14.2919,
 14.1913,14.4068,14.103 ,14.1655,14.2691,14.0911,14.1219,14.239 ,14.0463,
 14.    ,14.1774,14.2121,13.9223,13.812 ,14.0537]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 13.6977
Minimum found at epsilon size: 0.099


**------Node Optimization For Insert-----------**

**---------------Run 1-----------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

# Define leaf size range
leaf_size_range_start = 100
leaf_size_range_end = 2500  # Ensure inclusive range
leaf_size_step = 100

leaf_size_rebuld = 2250
epsilon_rebuild = 0.099

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuld, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]
    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    kdtree.leaf_size = leaf_size
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 100): 100%|██████████| 100/100 [00:00<00:00, 101.74it/s]


-------------------------------------------------------------
Build Time: 10.8314 sec, Search Time: 1.8347 sec
Leaf Size: 100, Total Time: 12.6661 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 200): 100%|██████████| 100/100 [00:00<00:00, 103.55it/s]


-------------------------------------------------------------
Build Time: 10.7606 sec, Search Time: 1.7950 sec
Leaf Size: 200, Total Time: 12.5556 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 11.1554 sec, Search Time: 1.7991 sec
Leaf Size: 2,300, Total Time: 12.9546 sec

24: === Testing with Leaf Size: 2,400, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,400): 100%|██████████| 100/100 [00:00<00:00, 102.46it/s]


-------------------------------------------------------------
Build Time: 11.0686 sec, Search Time: 1.8289 sec
Leaf Size: 2,400, Total Time: 12.8975 sec

25: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 100/100 [00:00<00:00, 105.00it/s]

-------------------------------------------------------------
Build Time: 10.5631 sec, Search Time: 1.8162 sec
Leaf Size: 2,500, Total Time: 12.3793 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 100, Build Time: 10.8314 sec, Search Time: 1.8347 sec, Total Time: 12.6661 sec
Leaf Size: 200, Build Time: 10.7606 sec, Search Time: 1.7950 sec, Total Time: 12.5556 sec
Leaf Size: 300, Build Time: 10.7251 sec, Search Time: 1.8151 sec, Total Time: 12.5402 sec
Leaf Size: 400, Build Time: 10.9716 sec, Search Time: 1.8722 sec, Total Time: 12.8437 sec
Leaf Size: 500, Build Time: 11.0207 sec, Search Time: 1.7967 sec, Total Time: 12.8175 sec
Leaf Size: 600, Build Time: 10.7997 sec, Search Time: 1.7928 sec, Total Time: 12.5925 sec
Leaf Size: 700, Build Time: 10.7802 sec, Search Time: 1.7847 sec, Total Time: 12.5649 sec
Leaf Size: 800, Build Time: 10.9490 sec, Search Time: 1.8112 sec, Total Time: 12.7603 sec
Leaf Size: 900, Build Time: 10.6467 sec, Search Time: 1.8958 sec, Total Time: 12.542

**-------------Run 2-----------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

# Define leaf size range
leaf_size_range_start = 100
leaf_size_range_end = 2500  # Ensure inclusive range
leaf_size_step = 100

leaf_size_rebuld = 2250
epsilon_rebuild = 0.099

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuld, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]
    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    kdtree.leaf_size = leaf_size
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 100): 100%|██████████| 100/100 [00:00<00:00, 105.63it/s]


-------------------------------------------------------------
Build Time: 10.9095 sec, Search Time: 1.7820 sec
Leaf Size: 100, Total Time: 12.6915 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 200): 100%|██████████| 100/100 [00:00<00:00, 104.34it/s]


-------------------------------------------------------------
Build Time: 10.8041 sec, Search Time: 1.8252 sec
Leaf Size: 200, Total Time: 12.6292 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 10.7958 sec, Search Time: 1.7442 sec
Leaf Size: 2,300, Total Time: 12.5400 sec

24: === Testing with Leaf Size: 2,400, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,400): 100%|██████████| 100/100 [00:00<00:00, 106.35it/s]


-------------------------------------------------------------
Build Time: 10.8322 sec, Search Time: 1.7743 sec
Leaf Size: 2,400, Total Time: 12.6064 sec

25: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 100/100 [00:00<00:00, 107.85it/s]

-------------------------------------------------------------
Build Time: 10.8100 sec, Search Time: 1.7647 sec
Leaf Size: 2,500, Total Time: 12.5747 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 100, Build Time: 10.9095 sec, Search Time: 1.7820 sec, Total Time: 12.6915 sec
Leaf Size: 200, Build Time: 10.8041 sec, Search Time: 1.8252 sec, Total Time: 12.6292 sec
Leaf Size: 300, Build Time: 10.6850 sec, Search Time: 1.7539 sec, Total Time: 12.4389 sec
Leaf Size: 400, Build Time: 10.8133 sec, Search Time: 1.7568 sec, Total Time: 12.5701 sec
Leaf Size: 500, Build Time: 10.7758 sec, Search Time: 1.7567 sec, Total Time: 12.5326 sec
Leaf Size: 600, Build Time: 10.7061 sec, Search Time: 1.7842 sec, Total Time: 12.4904 sec
Leaf Size: 700, Build Time: 10.9116 sec, Search Time: 1.7742 sec, Total Time: 12.6858 sec
Leaf Size: 800, Build Time: 10.8405 sec, Search Time: 1.7880 sec, Total Time: 12.6285 sec
Leaf Size: 900, Build Time: 10.7151 sec, Search Time: 1.8014 sec, Total Time: 12.516

**----Node Size Optimization For Insertion---------**

In [ ]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array(
    [ 100, 200, 300, 400, 500, 600, 700, 800, 900,1000,1100,1200,1300,1400,
 1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500]
)

# Total times from two runs (or versions)
total_times_1 = np.array(
    [12.6661,12.5556,12.5402,12.8437,12.8175,12.5925,12.5649,12.7603,12.5426,
 12.528 ,12.5421,12.7895,12.7391,12.8196,12.5516,12.637 ,12.714 ,12.6114,
 12.5816,12.7974,12.4308,12.7699,12.9546,12.8975,12.3793]
)

total_times_2 = np.array(
    [12.6915,12.6292,12.4389,12.5701,12.5326,12.4904,12.6858,12.6285,12.5165,
 12.5908,12.4073,12.554 ,12.5474,12.6551,12.5763,12.4004,12.7256,12.4319,
 12.6285,12.5702,12.4939,12.7568,12.54  ,12.6064,12.5747]
)

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 12.4624
Minimum found at leaf size: 2100


**------------Epsilon Optimization for Insertion-------------**

**-------------------Run 1---------------------------**

In [11]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100

# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.07
epsilon_size_step = 0.005

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Readjust for the insertion
    data_size, query_size = test_combinations[0]
    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    kdtree.leaf_size = leaf_size_insert
    kdtree.epsilon = epsilon

    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon Size {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon Size 0.0): 100%|██████████| 100/100 [00:00<00:00, 118.30it/s]


-------------------------------------------------------------
Build Time: 12.5147 sec, Search Time: 1.6459 sec
Epsilon: 0.0, Total Time: 14.1606 sec

2: === Testing with Epsilon: 0.005, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon Size 0.005): 100%|██████████| 100/100 [00:00<00:00, 116.33it/s]


-------------------------------------------------------------
Build Time: 11.9681 sec, Search Time: 1.6540 sec
Epsilon: 0.005, Total Time: 13.6221 sec

3: === Testing with Epsilon: 0.01, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 11.5902 sec, Search Time: 1.9401 sec
Epsilon: 0.065, Total Time: 13.5304 sec

15: === Testing with Epsilon: 0.07, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon Size 0.07): 100%|██████████| 100/100 [00:00<00:00, 106.39it/s]


-------------------------------------------------------------
Build Time: 11.3153 sec, Search Time: 1.7396 sec
Epsilon: 0.07, Total Time: 13.0549 sec

16: === Testing with Epsilon: 0.075, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon Size 0.075): 100%|██████████| 100/100 [00:00<00:00, 105.05it/s]

-------------------------------------------------------------
Build Time: 11.4487 sec, Search Time: 1.7292 sec
Epsilon: 0.075, Total Time: 13.1779 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 12.5147 sec, Search Time: 1.6459 sec, Total Time: 14.1606 sec
Epsilon: 0.005, Build Time: 11.9681 sec, Search Time: 1.6540 sec, Total Time: 13.6221 sec
Epsilon: 0.01, Build Time: 11.6634 sec, Search Time: 1.6664 sec, Total Time: 13.3299 sec
Epsilon: 0.015, Build Time: 11.7914 sec, Search Time: 1.7120 sec, Total Time: 13.5033 sec
Epsilon: 0.02, Build Time: 11.6757 sec, Search Time: 1.6742 sec, Total Time: 13.3499 sec
Epsilon: 0.025, Build Time: 11.5595 sec, Search Time: 1.6992 sec, Total Time: 13.2587 sec
Epsilon: 0.03, Build Time: 11.5900 sec, Search Time: 1.6822 sec, Total Time: 13.2722 sec
Epsilon: 0.035, Build Time: 11.4627 sec, Search Time: 1.6932 sec, Total Time: 13.1559 sec
Epsilon: 0.04, Build Time: 11.9085 sec, Search Time: 1.6879 sec, Total Time: 13.5964 sec

**----------------Run 2-------------------------**

In [13]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100

# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.07
epsilon_size_step = 0.005

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Readjust for the insertion
    data_size, query_size = test_combinations[0]
    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    kdtree.leaf_size = leaf_size_insert
    kdtree.epsilon = epsilon

    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon Size {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon Size 0.0): 100%|██████████| 100/100 [00:00<00:00, 114.65it/s]


-------------------------------------------------------------
Build Time: 12.3722 sec, Search Time: 1.6680 sec
Epsilon: 0.0, Total Time: 14.0402 sec

2: === Testing with Epsilon: 0.005, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon Size 0.005): 100%|██████████| 100/100 [00:00<00:00, 116.94it/s]


-------------------------------------------------------------
Build Time: 11.8296 sec, Search Time: 1.6489 sec
Epsilon: 0.005, Total Time: 13.4785 sec

3: === Testing with Epsilon: 0.01, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 11.4483 sec, Search Time: 1.7188 sec
Epsilon: 0.065, Total Time: 13.1671 sec

15: === Testing with Epsilon: 0.07, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon Size 0.07): 100%|██████████| 100/100 [00:00<00:00, 108.05it/s]


-------------------------------------------------------------
Build Time: 11.5195 sec, Search Time: 1.7170 sec
Epsilon: 0.07, Total Time: 13.2364 sec

16: === Testing with Epsilon: 0.075, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon Size 0.075): 100%|██████████| 100/100 [00:00<00:00, 105.20it/s]

-------------------------------------------------------------
Build Time: 11.5196 sec, Search Time: 1.7586 sec
Epsilon: 0.075, Total Time: 13.2782 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 12.3722 sec, Search Time: 1.6680 sec, Total Time: 14.0402 sec
Epsilon: 0.005, Build Time: 11.8296 sec, Search Time: 1.6489 sec, Total Time: 13.4785 sec
Epsilon: 0.01, Build Time: 11.8087 sec, Search Time: 1.6714 sec, Total Time: 13.4801 sec
Epsilon: 0.015, Build Time: 11.6696 sec, Search Time: 1.6456 sec, Total Time: 13.3152 sec
Epsilon: 0.02, Build Time: 11.8171 sec, Search Time: 1.6987 sec, Total Time: 13.5158 sec
Epsilon: 0.025, Build Time: 11.9534 sec, Search Time: 1.7004 sec, Total Time: 13.6537 sec
Epsilon: 0.03, Build Time: 11.4647 sec, Search Time: 1.6724 sec, Total Time: 13.1371 sec
Epsilon: 0.035, Build Time: 11.9442 sec, Search Time: 1.7029 sec, Total Time: 13.6470 sec
Epsilon: 0.04, Build Time: 11.4038 sec, Search Time: 1.6790 sec, Total Time: 13.0827 sec

**------Epsilon Optimization For Insertion----**

In [14]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array(
     [0.   ,0.005,0.01 ,0.015,0.02 ,0.025,0.03 ,0.035,0.04 ,0.045,0.05 ,0.055,
 0.06 ,0.065,0.07 ,0.075]
)

# Total times from two runs (or versions)
total_times_1 = np.array(
    [14.1606,13.6221,13.3299,13.5033,13.3499,13.2587,13.2722,13.1559,13.5964,
 13.459 ,13.3313,13.1443,13.5414,13.5304,13.0549,13.1779]
)

total_times_2 = np.array(
    [14.0402,13.4785,13.4801,13.3152,13.5158,13.6537,13.1371,13.647 ,13.0827,
 13.3291,13.1411,13.4511,13.3829,13.1671,13.2364,13.2782]
)

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 13.1456
Minimum found at epsilon size: 0.07
